In [1]:
%reload_ext autoreload
%autoreload 2

# OpenAI

In [ ]:
#%%writefile "../openaigpt.py"
import os, sys

sys.path.append(os.path.expanduser("~/.django") )
import my_config
from my_config import *

from openai import OpenAI

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key=my_config.OPENAPI_KEY
)

'''response = client.completions.create(
    model="llama3.2",
    prompt="____",
    max_tokens=256
)

print(response.choices[0].text)
'''

completion = client.chat.completions.create(
  model = "llama3.2",
  messages=[
    {"role": "system"    , "content": "You are a helpful assistant."},
    {"role": "user"      , "content": "Who won the world series in 2020?"},
    {"role": "assistant" , "content": "The LA Dodgers won in 2020."},
    {"role": "user"      , "content": "Where was it played?"}
  ]
)

print(completion.choices[0].message)

# OLLAMA

In [7]:
%%writefile "../olgpt.py"
import sys, os, ollama, torch, logging,datetime
from ollama import Client
from mangorest.mango import webapi
sys.path.append(os.path.expanduser("~/.django") )
from openai import OpenAI
    
logger = logging.getLogger( "geoapp" )

device = "cpu"
if (torch.cuda.is_available() ):
    device = "cuda"


OLLAMA_HOST= 'http://10.0.0.29:11434/v1'

if (os.path.exists(os.path.expanduser("~/.django/my_config.py"))):
    import my_config
    try:
        from my_config import OLLAMA_HOST
    except:
        pass

client = OpenAI(
    base_url = OLLAMA_HOST,
    api_key=my_config.OPENAPI_KEY
)

OLLAMA = Client(host=OLLAMA_HOST)
logger.info(f"OLLAMA_HOST => {OLLAMA_HOST}")
#--------------------------------------------------------------------------------------------------------    
@webapi("/gpt/generate/")
def ollma_generate(request=None, model="llama3|mistral", prompt="", stream=True,**kwargs):
    logger.info(f"Prepping: {datetime.datetime.now()}")
    model = model.split("|")[0].strip()
    response = OLLAMA.generate(model=model, prompt=prompt, stream=stream)
    # Stream response
    ret = ""
    for chunk in response:
        data = chunk["response"]
        ret += data
        
    logger.debug(ret)
    return ret

#--------------------------------------------------------------------------------------------------------    
@webapi("/gpt/openai/")
def openai(request=None, model="llama3.2|mistral", host="", prompt="", messages="", **kwargs):

    client = OpenAI( base_url = host or OLLAMA_HOST, api_key  = my_config.OPENAPI_KEY )

    tmsg = [
                {"role": "system"    , "content": "You are a helpful assistant."},
                {"role": "user"      , "content": prompt or "Where was it played?"}
    ]
    
    messages = messages or tmsg
    model    = model.split("|")[0]
    
    logger.info(f"{model} : {messages}")
    
    completion = client.chat.completions.create(
        model = "llama3.2",
        messages= tmsg
        )

    ret = completion.choices[0].message
    
    return ret.content




2024-10-31 22:23:28,440 geoapp INFO: OLLAMA_HOST => http://127.0.0.1:11434/v1
2024-10-31 22:23:28,461 geoapp INFO: llama3.2 : [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'who won 1996 election'}]
2024-10-31 22:23:36,799 httpx INFO: HTTP Request: POST http://127.0.0.1:11434/v1/chat/completions "HTTP/1.1 200 OK"


"The 1996 United States presidential election was held on November 5, 1996. The winners of the presidential election were:\n\n* Bill Clinton (Democratic Party) - re-elected as President\n* Bob Dole (Republican Party) - lost to Bill Clinton in the general election\n\nHowever, it's worth noting that Al Gore won the popular vote but lost the electoral college vote to George W. Bush in some states in the 1996 presidential election"

In [ ]:
#ollma_generate(prompt="whos is the chief data scientist of Space Lockheed Martin", stream=True)   
openai(prompt="who won 1996 election")

# Experiments

In [11]:
import json
fg = '''  
messages=[
    {"role": "system"    , "content": "You are a helpful assistant."},
    {"role": "user"      , "content": "Who won the world series in 2020?"},
    {"role": "assistant" , "content": "The LA Dodgers won in 2020."},
    {"role": "user"      , "content": "Where was it played?"}
  ]
'''

#json.loads(fg)